In [3]:
import openeo
# import ProcessBuilder functions
from openeo.processes import ProcessBuilder

# Establish a connection to openeo
connection = openeo.connect("openeofed.dataspace.copernicus.eu")
connection.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeofed.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [10]:
exCube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 4.312323, "south":  50.922796, "east": 4.335497, "north": 50.934860},
    temporal_extent = ["2021-02-01", "2021-04-30"],
    bands=["B04", "B03", "B02", "B08"]
)

# Processes

## Terminology

In openEO, a process is an operation that performs a specific task on a set of parameters and returns a result.
A process is similar to a function in common programming languages, and likewise, multiple processes can be combined or chained together into new, more complex operations.

A pre-defined process is a process provided out of the box by a given back-end. These are often the [centrally defined openEO processes](https://openeo.org/documentation/1.0/processes.html), such as common mathematical sum, divide, sqrt, …), statistical (mean, max, …) and image processing (mask, apply_kernel, …) operations. 

Processes can be combined into a larger pipeline, parameterized and stored on the back-end as a so called [user-defined process](https://open-eo.github.io/openeo-python-client/udp.html#user-defined-processes). This allows you to build a library of reusable building blocks that can be be inserted easily in multiple other places

In [4]:
process_ids = [process["id"] for process in connection.list_processes()]

In [5]:
process_ids

['_cwl_dummy_stac',
 'absolute',
 'add',
 'add_dimension',
 'aggregate_spatial',
 'aggregate_temporal',
 'aggregate_temporal_period',
 'all',
 'and',
 'any',
 'apply',
 'apply_dimension',
 'apply_kernel',
 'apply_neighborhood',
 'apply_polygon',
 'arccos',
 'arcosh',
 'arcsin',
 'arctan',
 'arctan2',
 'array_append',
 'array_apply',
 'array_concat',
 'array_contains',
 'array_create',
 'array_element',
 'array_filter',
 'array_find',
 'array_interpolate_linear',
 'array_labels',
 'arsinh',
 'artanh',
 'between',
 'ceil',
 'clip',
 'constant',
 'cos',
 'cosh',
 'count',
 'date_shift',
 'dimension_labels',
 'discard_result',
 'divide',
 'drop_dimension',
 'e',
 'eq',
 'exp',
 'extrema',
 'filter_bands',
 'filter_bbox',
 'filter_labels',
 'filter_spatial',
 'filter_temporal',
 'first',
 'fit_class_catboost',
 'floor',
 'get_geometries',
 'gt',
 'gte',
 'if',
 'inspect',
 'int',
 'is_nan',
 'is_nodata',
 'is_valid',
 'last',
 'linear_scale_range',
 'ln',
 'load_collection',
 'load_disk_dat

In [6]:
connection.describe_process("add")

{'categories': ['math'],
 'deprecated': False,
 'description': 'Sums up the two numbers `x` and `y` (*`x + y`*) and returns the computed sum.\n\nNo-data values are taken into account so that `null` is returned if any element is such a value.\n\nThe computations follow [IEEE Standard 754](https://ieeexplore.ieee.org/document/8766229) whenever the processing environment supports it.',
 'examples': [{'arguments': {'x': 5, 'y': 2.5}, 'returns': 7.5},
  {'arguments': {'x': -2, 'y': -4}, 'returns': -6},
  {'arguments': {'x': 1, 'y': None}, 'returns': None}],
 'experimental': False,
 'federation:backends': ['cdse', 'terrascope'],
 'id': 'add',
 'links': [{'href': 'http://mathworld.wolfram.com/Sum.html',
   'rel': 'about',
   'title': 'Sum explained by Wolfram MathWorld'},
  {'href': 'https://ieeexplore.ieee.org/document/8766229',
   'rel': 'about',
   'title': 'IEEE Standard 754-2019 for Floating-Point Arithmetic'}],
 'parameters': [{'description': 'The first summand.',
   'name': 'x',
   'schema': {'type': ['number', 'null']}},
  {'description': 'The second summand.',
   'name': 'y',
   'schema': {'type': ['number', 'null']}}],
 'returns': {'description': 'The computed sum of the two numbers.',
  'schema': {'type': ['number', 'null']}},
 'summary': 'Addition of two numbers'}

Most of the important pre-defined processes are covered directly by methods on classes like DataCube or VectorCube.

In [11]:
arguments= {
    "data": [1, 3, -1]
}
res = exCube.process("mean", arguments)

In [18]:
res = exCube.process("ndvi", data=exCube)

#Equivalent expression using the THIS expression

from openeo.rest.datacube import THIS

res = exCube.process("ndvi", data=THIS)

#Multiple expressions can be combined

res = (
    exCube
        .process("filter_bands", data=THIS)
        .process("ndvi", data=THIS)
)

The openEO Python Client Library supports several ways to specify the desired callback for these functions:

* Callback as string
* Callback as a callable
* Callback as PGNode

In [20]:
# Take the absolute value of each pixel - String execution
exCube.apply("absolute")

# callback execution
def transform(x):
    return x * 2 + 3

exCube.apply(transform)

exCube.apply(lambda x: x * 2 + 3) #analogously to transform function

The argument that will be passed to all these callback functions is a ProcessBuilder instance. This is a helper object with predefined methods for all standard openEO processes, allowing to use an object oriented coding style to define the callback. 

In [ ]:
from openeo.processes import ProcessBuilder

def avg(data: ProcessBuilder):
    return data.mean()

cube.reduce_dimension(reducer=avg, dimension="t")